# REINFORCEMENT LEARNING WITH DEEP Q-LEARNING

## DEEP Q-LEARNING OVERVIEW

The logic of the Deep _Q-learning algorithm_ is very similar to _Q-learning_, but in this case, the policy or _Q-table_ is not learned directly through the _Q-function_. Instead, it is learned by a neural network.

A neural network is defined such that, given a state, it returns the best Q(s, a) value across all possible actions, selecting only those with the highest Q(s, a) values. The network consists of:

- **Input Layer**: Represents the necessary neurons to encode the agent's current state (In 2D grid example, x and y coordinates, so 2 neurons).
- **Output Layer**: Encodes the actions the agent can take in the environment (In 2D grid example, 4 actions: up, down, left, and right).
- **Intermediate Layers**: Determined based on the complexity of the system.

## SETUP

In [ ]:
import os
import sys
import pandas as pd
from copy import deepcopy

pd.set_option("display.max_columns", None)

# Add the parent directory (where "modules" is located) to the Python path
notebook_dir = os.getcwd()
parent_dir = os.path.dirname(notebook_dir)
sys.path.append(parent_dir)

from modules import environment_creation_functions as ecf
from modules import environment as env
from modules import deep_q_learner as dqlearn
from modules import deep_q_learning_env_agent_execution as dqae

## LOAD ENVIRONMENT

Initial state of the agent is represented in green color, the final or destination state in red and barrier in black.

In [ ]:
# create environment
dim, grid_matrix_positions, barriers, src, dest, reward_matrix  = ecf.create_environment(False, 20, 100)
environment = env.Environment(reward_matrix, list(src), list(dest)) # convert states from tuple into list

# visualize created environment
%matplotlib inline
ecf.visualize_environment(dim, barriers, src, dest)

## DEEP Q-LEARNING ALGORITHM

### LEARNING PROCESS

The learning process in the Deep Q-learning algorithm does not occur per action, as it does in the `Q-learning` or `SARSA` algorithms. Instead, it occurs over entire episodes, where the algorithm takes actions and stores them in memory. This memory is called `Experience Replay`, and for each action, it records:

- the **current state**,
- the **action taken**,
- the **reward** obtained,
- the **next state**, and
- whether or not **it is a terminal state**.

Once the memory is filled, a set of actions is selected (i.e.: randomly), and a model is trained to adjust the weights, allowing the network to learn from each action.

- Algorithm pseudocode:

   ```
   # Initialize Memory (Experience Replay) : Current State | Taken Action | Reward | Next State | Is final state?
   # Set memory size i.e. mem_size = 200
   # Neuronal Network initialization
   For each episode do:
        s = initial state
        actions_per_episode = 0
        memory = empty
        while (s != not final) and (actions_per_episode < mem_size) do:
            a = Choose action. Cases:
                - random(action) if random() >= exploration_rate 
                - Predict (Neuronal network): Q_predict = max_a Q(St,a)  if random() < exploration_rate  
            Execute action a           
            R(S_t, a) = reward
            S_t = S_t+1 # New state
            save_in_memory = S_t, a, R(S_t, a), S_t+1, is_final_state?
        batch = select actions from memory (randomly)
        for each element in batch do:
            Predict (Neuronal network) = Q_predict(S_t, a)            
            Q_obj(S_t, a) =  R(S_t, a)  + (gamma . max_a'Q(S_t+1_, a'))     
            Train Neuronal Netwok (X= S_t, Y = Q_obj(S_t, a), loss = MSE) 
                Loss => 1/2 Sum(TD(S_t,a)^2) # temporal difference error
        end
    end
    ```

### SHORT-TERM FOCUS

In [ ]:
# Apply agent deep q-learning and print results
dqae.run_agent(learner=dqlearn.DeepQLearner,
          num_episodes=30, 
          discount_factor=0.1, # Near 0, learn to move to the next most rewarding state
          # Aim to obtain 95% explotation, during the learning process is adjusted.
          # Starts from exploration and gradually goes to explotation.
          ratio_explotacion=0.95, # Greedy control
          max_steps=500, # TODO HOW TO DETERMINE BEST MAX_STEPS
          verbose=True)

### LONG-TERM FOCUS

In [ ]:
# Apply agent deep q-learning and print results
dqae.run_agent(learner=dqlearn.DeepQLearner,
          num_episodes=30,
          discount_factor=0.9, # Near 1, move with a view to maximizing the final reward
          # Aim to obtain 95% explotation, during the learning process is adjusted.
          # Starts from exploration and gradually goes to explotation.
          ratio_explotacion=0.95,   # Greedy control
          max_steps=500, # TODO HOW TO DETERMINE BEST MAX_STEPS
          verbose=True)